# Damage Assessment APP - Back-end Code


- [1.0 Google Address Reverse Geocoding API](#1.0-Google-Address-Reverse-Geocoding-API)
- [2.0 Google Image Street View API](#2.0-Google-Image-Street-View-API)
- [3.0 Zillow Data API](#3.0-Zillow-Data-API)


In [193]:
import pandas as pd
import requests

## 1.0 Google Address Reverse Geocoding API

### 1.1 Input: 

In [2]:
#  Lattitude and Longitude from extracted from picture
#  Input format: turple or list 
#  example: (37.79100833333333, -122.40064166666667) or
#           [37.79100833333333, -122.40064166666667]

lat_long_raw = (37.79100833333333, -122.40064166666667)

In [3]:
# number of addresses to return
n_address = 4

In [4]:
# API Key
# !!!SAVE YOUR API AS A CSV ON YOUR LOCAL DRIVE AND READ USING THE CODE BELOW, DO NOT EXPLICITLY USE
#    API KEY IN THIS BOOK!!!
api_key = pd.read_csv('/Users/kaizhao/Documents/project_API/google_api.csv', header=None).loc[0,0]

### 1.2 Data Mining Through Google API

In [11]:
# Generate Google API url
# Input: lat_long_raw & api_key (see above section 1.1 for instruction)
# Output: Google API url

def get_url_geocode(lat_long_raw, api_key):
    lat, long = lat_long_raw
    latlng = str(lat)+','+str(long)
    url_geocode = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={latlng}&key={api_key}'
    return url_geocode

In [14]:
# Generate Google API url
# Input: lat_long_raw & api_key (see above section 1.1 for instruction)
#        n_address: integer - number of addresses to return
# Output: list of dictionary of addresses


def get_address(lat_long_raw, api_key, n_address):
    
    url = get_url_geocode(lat_long_raw, api_key)

    res = requests.get(url)
    if res.status_code == 200:
        print(f'Status code: {res.status_code}')
        res_jas = res.json()
    
        address_list = []
        for i in range(n_address):
            address={}
            address['full_address'] = res_jas['results'][i]['formatted_address']
            address['street_number'] = address['full_address'].split(', ')[0]
            address['city'] = address['full_address'].split(', ')[1]
            address['state'] = address['full_address'].split(', ')[2].split(' ')[0]
            address['zip_code'] = address['full_address'].split(', ')[2].split(' ')[1]
            address['lat'] = res_jas['results'][i]['geometry']['location']['lat']
            address['lng'] = res_jas['results'][i]['geometry']['location']['lng']
            address['place_id'] = res_jas['results'][i]['place_id']
            address_list.append(address)
    else:
        print('Unexpected error: Check latitude, longitude format and Api Key')
    
    return address_list

### 1.3 Outout

In [17]:
address_list = get_address(lat_long_raw, api_key, n_address)

Status code: 200


In [18]:
# address_list

[{'full_address': '0290011, San Francisco, CA 94104, United States',
  'street_number': '0290011',
  'city': 'San Francisco',
  'state': 'CA',
  'zip_code': '94104',
  'lat': 37.7909707,
  'lng': -122.4005609,
  'place_id': 'ChIJ2UojPeqBhYAR6RgupRitcQM'},
 {'full_address': '8 Sansome St, San Francisco, CA 94104, USA',
  'street_number': '8 Sansome St',
  'city': 'San Francisco',
  'state': 'CA',
  'zip_code': '94104',
  'lat': 37.7908097,
  'lng': -122.40047,
  'place_id': 'ChIJXdeibWKAhYARyaiiGHmYg50'},
 {'full_address': 'Bush St & Sansome St, San Francisco, CA 94104, USA',
  'street_number': 'Bush St & Sansome St',
  'city': 'San Francisco',
  'state': 'CA',
  'zip_code': '94104',
  'lat': 37.7910704,
  'lng': -122.4009497,
  'place_id': 'ChIJITZbDGKAhYARvTKTAY1KjOk'},
 {'full_address': '54 Sansome St, San Francisco, CA 94104, USA',
  'street_number': '54 Sansome St',
  'city': 'San Francisco',
  'state': 'CA',
  'zip_code': '94104',
  'lat': 37.7909835,
  'lng': -122.4007399,
  'pla

### 1.4 Validation

In [ ]:
# allow the user to validate the 4 addresses output by the reverse geocoding: option in 0-3
# if none of the addressed match (option == 4), user need to manually input address
# input outside 0-4 will lead to an ERROR message

def address_select():
    n = None
    while n not in range (5):
        n = int(input('Selection: '))
        if n in range(4):
            address = address_list[n]['full_address']
            print(address)
            return address
        elif n == 4:
            street_number = input('Input Street Address: ')
            city = input('Input city: ')
            state = input('Input State: ')
            zip_code = input('Input zip code: ')
            address = street_number + ', ' + city + ', ' + state + ' ' + zip_code 
            print(address)
            return address
        else:
            print('Please input valide selection')

In [200]:
address = address_select()

Selection: 4
Input Street Address: 369 Churchill Ave
Input city: Palo Alto
Input State: CA
Input zip code: 94301
369 Churchill Ave, Palo Alto, CA 94301


## 2.0 Google Image Street View API


In [201]:
from io import open as iopen

In [202]:
# url generated based on validated address
url_image = f'https://maps.googleapis.com/maps/api/streetview?size=\
400x400&location={address}&key={api_key}'

In [203]:
# Streetview image extracted based on the url generated above
filename = address.replace(' ', '_').replace(',', '')
res_image = requests.get(url_image)
if res_image.status_code == 200:
    print(f'Status code: {res_image.status_code}')
    with iopen(filename+'.jpg', 'wb') as file:
        file.write(res_image.content)
else:
    print('Unexpected error')

Status code: 200


## 3.0 Zillow Data API

In [180]:
from bs4 import BeautifulSoup

### 3.1 Input

In [181]:
# zillow web service API
ZWSID = pd.read_csv('/Users/kaizhao/Documents/project_API/zillow_api.csv', header=None).loc[0,0]

### 3.2 Obtain Property Info

In [182]:
# Generate street address and zip code input based on validated google reverse geocoding
street_address = address.split(', ')[0].replace(' ', '+')
zip_code = address.split(', ')[2].split(' ')[1]

In [183]:
# Generate api url based on the street address and zip code above
z_info_url = f'http://www.zillow.com/webservice/GetDeepSearchResults.htm?\
zws-id={ZWSID}&address={street_address}&citystatezip={zip_code}'

In [184]:
# collecting property information
res_z_info = requests.get(z_info_url)
soup = BeautifulSoup(res_z_info.content, ['lxml-xml'])
property_info = {}
property_info['zpid'] = soup.find('zpid').text
property_info['current_value'] = int(soup.find('amount').text)
property_info['last_sold'] = int(soup.find('lastSoldPrice').text)
property_info['last_sold_date'] = soup.find('lastSoldDate').text
property_info['property_type'] = soup.find('useCode').text
property_info['year_built'] = int(soup.find('yearBuilt').text)
property_info['bedrooms'] = int(soup.find('bedrooms').text)
property_info['bathrooms'] = float(soup.find('bathrooms').text)
property_info['sqft'] = int(soup.find('finishedSqFt').text)
property_info['lot_size'] = int(soup.find('lotSizeSqFt').text)

In [185]:
property_info

{'zpid': '19508108',
 'current_value': 2663108,
 'last_sold': 925000,
 'last_sold_date': '03/15/2002',
 'property_type': 'SingleFamily',
 'year_built': 1954,
 'bedrooms': 3,
 'bathrooms': 2.0,
 'sqft': 1598,
 'lot_size': 8611}

### 3.3 Obtain Price Chart

In [186]:
# url parameters
unit = 'dollar' # 'percent' or 'dollar'
duration = '5year' # '1year', '5year', or '10year'. default = '1year'
zpid = property_info['zpid'] # from section 3.2

# url address
z_chart_url = f'http://www.zillow.com/webservice/GetChart.htm?\
zws-id={ZWSID}&unit-type={unit}&zpid={zpid}&width=600&height=300&chartDuratoin={duration}'

In [192]:
# obtain url to embed in the web-app
res_z_chart = requests.get(z_chart_url)
soup = BeautifulSoup(res_z_chart.content, ['lxml-xml'])
chart_url = soup.find('url').text

In [188]:
# url for embedding web_app
chart_url

'http://www.zillow.com/app?chartDuration=1year&chartType=partner&height=300&page=webservice%2FGetChart&service=chart&width=600&zpid=19508108'

### 3.4 Obtain Zillow Image
> Zillow Image usually have better quality but are usually legally protected

In [189]:
# url address
z_image_url = f'http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?\
zws-id={ZWSID}&zpid={zpid}'

In [190]:
# obtain url to incorporate in the web-app
res_z_image = requests.get(z_image_url)
print(f'Status code: {res_z_image.status_code}')
soup = BeautifulSoup(res_z_image.content, ['lxml-xml'])
image_url = soup.find_all('url')

Status code: 200


In [191]:
image_url

[]